In [26]:
# Baseline Experiments: VLM и LLM Evaluation

"""В этом ноутбуке мы:
- Извлекаем ингредиенты через VLM (LLaVAVision)
- Генерируем рецепты через LLM (Mistral)
- Проверяем рецепты на диетические ограничения и уровни сложности
- Считаем latency, F1, Excess, токены
- Сохраняем результаты в отчёт """

'В этом ноутбуке мы:\n- Извлекаем ингредиенты через VLM (LLaVAVision)\n- Генерируем рецепты через LLM (Mistral)\n- Проверяем рецепты на диетические ограничения и уровни сложности\n- Считаем latency, F1, Excess, токены\n- Сохраняем результаты в отчёт '

In [21]:
import os
import json
import time
import pandas as pd
from ml.models.baseline import LLaVAVision

In [22]:
def compute_precision_recall_f1(predicted, reference):
    predicted_set = set([p.lower() for p in predicted])
    reference_set = set([r.lower() for r in reference])

    tp = len(predicted_set & reference_set)
    fp = len(predicted_set - reference_set)
    fn = len(reference_set - predicted_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    return f1

def compute_excess(predicted, reference):
    reference_set = set([r.lower() for r in reference])
    predicted_set = set([p.lower() for p in predicted])
    excess = predicted_set - reference_set
    return len(excess) / len(predicted_set) if predicted_set else 0.0

In [88]:
def evaluate_vlm(eval_file="ml/tracing/vlm_eval_cases.json"):
    with open(eval_file, "r", encoding="utf-8") as f:
        eval_cases = json.load(f)

    vlm = LLaVAVision()
    results = []

    for idx, case in enumerate(eval_cases, start=1):
        image_path = case["image_path"]
        reference = case["reference_ingredients"]

        print(f"\n▶️ Тест #{idx}: {os.path.basename(image_path)}")

        start = time.time()
        pred = vlm.infer(image_path)
        latency = time.time() - start

        predicted = [ing["name"] if isinstance(ing, dict) else ing
                     for ing in pred.get("ingredients", [])]

        f1 = compute_precision_recall_f1(predicted, reference)
        excess = compute_excess(predicted, reference)

        results.append({
            "id": idx,
            "image": os.path.basename(image_path),
            "reference": reference,
            "predicted": predicted,
            "F1": round(f1, 3),
            "Excess": round(excess, 3),
            "Latency (сек)": round(latency, 3)
        })

    return results

In [1]:
results = evaluate_vlm()

NameError: name 'evaluate_vlm' is not defined

In [90]:
df = pd.DataFrame(results)
styled = (
    df.style
      .format({"F1": "{:.3f}", "Excess": "{:.3f}", "Latency (сек)": "{:.2f}"})
      .highlight_max(subset=["F1"], color="lightgreen")
      .highlight_min(subset=["Latency (сек)"], color="lightblue")
)
display(styled)

print("📊 Сводка по всем тестам")
print(f"Средний F1: {df['F1'].mean():.3f}")
print(f"Средний Excess: {df['Excess'].mean():.3f}")
print(f"Средний Latency: {df['Latency (сек)'].mean():.3f} сек")

,id,image,reference,predicted,F1,Excess,Latency (сек)
0,1,1.jpg,"['Сыр', 'Курица', 'Брокколи', 'Вода', 'Перец', 'Яйца']","['брокколи', 'курица', 'сыр', 'вода']",0.800,0.000,51.71
1,2,2.jpg,"['Сыр', 'Сметана', 'Сливочное масло', 'Творог', 'Молоко', 'Яйца']","['молоко', 'сыр', 'яйца']",0.667,0.000,62.29
2,3,3.jpg,"['Грецкие орехи', 'Яйца', 'Лосось', 'Авокадо', 'Грибы', 'Латук', 'Киви', 'Яблоки', 'Апельсины']","['оливки', 'яйца', 'лосось', 'грибы', 'авокадо', 'салат', 'яблоки', 'апельсины', 'киви', 'грейпфрут', 'оливки', 'лосось', 'яйца', 'грибы', 'авокадо', 'салат', 'яблоки', 'апельсины', 'киви', 'грейпфрут']",0.737,0.300,85.29
3,4,4.jpg,"['Яблоки', 'Абрикосы', 'Морковь', 'Картофель', 'Мята']",[],0.000,0.000,184.15
4,5,5.jpg,['Яблоко'],['яблоко'],1.000,0.000,66.34


📊 Сводка по всем тестам
Средний F1: 0.641
Средний Excess: 0.060
Средний Latency: 89.958 сек


In [48]:
import os
import json
import time
import re
import requests
import pandas as pd
from dotenv import load_dotenv
from ml.models.baseline import MistralText

# Загружаем ключи
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
MISTRAL_URL = "https://api.mistral.ai/v1/chat/completions"
MISTRAL_MODEL = "mistral-medium"

llm = MistralText()

In [63]:
def clean_mistral_output(output: str) -> str:
    if not isinstance(output, str):
        return ""
    clean = re.sub(r"^```(?:json)?", "", output.strip(), flags=re.IGNORECASE | re.MULTILINE)
    clean = re.sub(r"```$", "", clean.strip(), flags=re.MULTILINE)
    clean = re.sub(r"\n\s*-\s*\n", "\n", clean)
    clean = re.sub(r"^\s*-\s*{", "{", clean, flags=re.MULTILINE)
    clean = re.sub(r'[\x00-\x1f\x7f]', ' ', clean)
    end = clean.rfind("}")
    if end != -1:
        clean = clean[:end+1]
    return clean.strip()

def check_with_mistral(recipes, dietary):
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }
    prompt = (
        "Ты проверяющий ассистент. Верни строго валидный JSON без Markdown.\n\n"
        f"Рецепты (JSON):\n{json.dumps(recipes, ensure_ascii=False, indent=2)}\n\n"
        f"Диетические ограничения: {dietary or 'нет'}\n\n"
        "Проверь:\n"
        "1) Нет ли в рецептах запрещённых ингредиентов.\n"
        "   Учитывай производные: если указано 'лактоза', то нельзя использовать молочные продукты.\n"
        "2) Есть ли среди рецептов все уровни сложности: легкое, среднее, сложное.\n\n"
        "Ответь ТОЛЬКО JSON следующей формы:\n"
        "{\n"
        '  "dietary_ok": true,\n'
        '  "difficulty_ok": true\n'
        "}\n"
    )
    payload = {
        "model": MISTRAL_MODEL,
        "messages": [
            {"role": "system", "content": "Ты проверяющий ассистент. Возвращай только JSON."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 256,
    }
    resp = requests.post(MISTRAL_URL, headers=headers, json=payload, timeout=60)

    if resp.status_code != 200:
        return {"error": f"Mistral API error: {resp.status_code}"}

    data = resp.json()
    content = (
        data.get("choices", [{}])[0]
        .get("message", {})
        .get("content", "")
        .strip()
    ) or data.get("choices", [{}])[0].get("text", "").strip()

    if not content:
        return {"error": "Empty response"}

    cleaned = clean_mistral_output(content)
    try:
        parsed = json.loads(cleaned)
    except Exception as e:
        return {"error": f"Invalid JSON: {e}", "raw_output": content}

    usage = data.get("usage", {}) or {}
    return {
        "dietary_ok": bool(parsed.get("dietary_ok", False)),
        "difficulty_ok": bool(parsed.get("difficulty_ok", False)),
        "usage": usage
    }

In [64]:
def run_tests(tests_path="ml/tracing/test_recipes.json", return_df=True):
    with open(tests_path, "r", encoding="utf-8") as f:
        test_cases = json.load(f)

    total = len(test_cases)
    passed_both = passed_diet = passed_diff = 0
    all_results = []

    for idx, case in enumerate(test_cases, start=1):
        print(f"\n▶️ Тест {idx}/{total}: ингредиенты={case['ingredients']}, диета={case.get('dietary', 'нет')}")

        start = time.time()
        recipes = llm.generate_recipe(
            case["ingredients"],
            dietary=case.get("dietary", "нет"),
            feedback=case.get("user_feedback", "нет")
        )
        gen_latency = round(time.time() - start, 3)

        if isinstance(recipes, dict) and "error" in recipes:
            print(f"  ❌ Ошибка генерации")
            all_results.append({
                "id": idx,
                "ingredients": case["ingredients"],
                "dietary": case.get("dietary", "нет"),
                "dietary_ok": None,
                "difficulty_ok": None,
                "gen_latency": gen_latency,
                "total_tokens": None,
                "error": recipes.get("error"),
                "recipes": []
            })
            continue

        if recipes is None:
            recipes = []
        elif isinstance(recipes, dict):
            recipes = [recipes]

        check_result = check_with_mistral(recipes, case.get("dietary", "нет"))
        dietary_ok = check_result.get("dietary_ok", False)
        difficulty_ok = check_result.get("difficulty_ok", False)
        usage = check_result.get("usage", {}) or {}

        if dietary_ok: passed_diet += 1
        if difficulty_ok: passed_diff += 1
        if dietary_ok and difficulty_ok: passed_both += 1

        print(f"  Проверка диеты: {'✅' if dietary_ok else '❌'}")
        print(f"  Проверка сложностей: {'✅' if difficulty_ok else '❌'}")

        all_results.append({
            "id": idx,
            "ingredients": case["ingredients"],
            "dietary": case.get("dietary", "нет"),
            "dietary_ok": dietary_ok,
            "difficulty_ok": difficulty_ok,
            "gen_latency": gen_latency,
            "total_tokens": usage.get("total_tokens"),
            "error": check_result.get("error") if "error" in check_result else None,
            "recipes": recipes
        })

    print(f"\n📊 Итог по {total} тестам: "
          f"Диета {passed_diet}/{total}, "
          f"Сложности {passed_diff}/{total}, "
          f"Оба {passed_both}/{total}")

    df = pd.DataFrame(all_results)
    return df if return_df else all_results

In [65]:
df = run_tests()

styled = (
    df.drop(columns=["recipes"])
      .style
      .format({"gen_latency": "{:.2f}"})
      .highlight_min(subset=["gen_latency"], color="lightblue")
      .highlight_max(subset=["gen_latency"], color="lightcoral")
)

display(styled)

print("📊 Средняя генерация (сек):", df["gen_latency"].dropna().mean().round(3))
print("📊 Средний total_tokens:", df["total_tokens"].dropna().mean().round(1))


▶️ Тест 1/10: ингредиенты=[{'name': 'картофель'}, {'name': 'молоко'}, {'name': 'курица'}], диета=лактоза
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 2/10: ингредиенты=[{'name': 'куриная грудка'}, {'name': 'рис'}, {'name': 'брокколи'}], диета=нет
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 3/10: ингредиенты=[{'name': 'макароны'}, {'name': 'фарш'}, {'name': 'чеснок'}], диета=нет
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 4/10: ингредиенты=[{'name': 'лосось'}, {'name': 'огурцы'}, {'name': 'помидоры'}], диета=рыба
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 5/10: ингредиенты=[{'name': 'фасоль'}, {'name': 'нут'}, {'name': 'курица'}], диета=нет
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 6/10: ингредиенты=[{'name': 'говядина'}, {'name': 'картофель'}, {'name': 'морковь'}], диета=нет
  Проверка диеты: ✅
  Проверка сложностей: ✅

▶️ Тест 7/10: ингредиенты=[{'name': 'яйца'}, {'name': 'сыр'}, {'name': 'помидоры'}], диета=нет
  Проверка диеты: ✅


,id,ingredients,dietary,dietary_ok,difficulty_ok,gen_latency,total_tokens,error
0,1,"[{'name': 'картофель'}, {'name': 'молоко'}, {'name': 'курица'}]",лактоза,True,True,19.65,2105,None
1,2,"[{'name': 'куриная грудка'}, {'name': 'рис'}, {'name': 'брокколи'}]",нет,True,True,16.33,1849,None
2,3,"[{'name': 'макароны'}, {'name': 'фарш'}, {'name': 'чеснок'}]",нет,True,True,26.47,2096,None
3,4,"[{'name': 'лосось'}, {'name': 'огурцы'}, {'name': 'помидоры'}]",рыба,True,True,20.99,2181,None
4,5,"[{'name': 'фасоль'}, {'name': 'нут'}, {'name': 'курица'}]",нет,True,True,21.84,1655,None
5,6,"[{'name': 'говядина'}, {'name': 'картофель'}, {'name': 'морковь'}]",нет,True,True,19.65,2474,None
6,7,"[{'name': 'яйца'}, {'name': 'сыр'}, {'name': 'помидоры'}]",нет,True,True,22.37,1565,None
7,8,"[{'name': 'макароны'}, {'name': 'сыр пармезан'}, {'name': 'говядина'}]",глютен,False,True,11.48,1240,None
8,9,"[{'name': 'огурцы'}, {'name': 'морковь'}, {'name': 'брокколи'}, {'name': 'курица'}]",нет,True,True,25.17,2206,None
9,10,"[{'name': 'сметана'}, {'name': 'грибы'}, {'name': 'картофель'}]",нет,True,True,25.41,2534,None


📊 Средняя генерация (сек): 20.936
📊 Средний total_tokens: 1990.5


In [66]:
rows = []
for _, row in df.iterrows():
    for rec in (row.get("recipes") or []):
        name = rec.get("name", "")
        ingredients = rec.get("ingredients", [])
        if isinstance(ingredients, dict):
            ingredients_list = [f"{k}: {v}" for k, v in ingredients.items()]
        else:
            ingredients_list = [i if isinstance(i, str) else i.get("name", "") for i in ingredients]
        rows.append({
            "id": row["id"],
            "dietary": row["dietary"],
            "recipe_name": name,
            "ingredients": ", ".join(ingredients_list),
            "difficulty": rec.get("difficulty", "")
        })

df_recipes = pd.DataFrame(rows)
display(df_recipes)

,id,dietary,recipe_name,ingredients,difficulty
0,1,лактоза,Картофельное пюре на воде с куриной подливкой,"картофель, курица (грудка), вода, соль, черный...",средне
1,1,лактоза,Запечённая курица с картофелем в фольге,"курица (бедро или голень), картофель, соль, па...",сложно
2,1,лактоза,Куриный бульон с картофелем (быстрый),"курица (крылышки или кости для бульона), карто...",легко
3,2,нет,Лёгкий салат с курицей и брокколи,"куриная грудка, брокколи, оливковое масло, сол...",легко
4,2,нет,Курица с рисом и брокколи на сковороде,"куриная грудка, рис, брокколи, растительное ма...",средне
5,2,нет,Запечённая курица с рисом и брокколи в горшочках,"куриная грудка, рис, брокколи, куриный бульон,...",сложно
6,3,нет,Чесночный соус с макаронами (аглио э олlio),"макароны, чеснок, оливковое масло, соль, перец...",средне
7,3,нет,Простые макароны с фаршем,"макароны, фарш, чеснок, вода, соль, растительн...",легко
8,3,нет,Запечённые макароны с фаршем и чесночной корочкой,"макароны, фарш, чеснок, томатная паста, вода, ...",сложно
9,4,рыба,Лёгкий салат из огурцов и помидоров с лимонной...,"огурцы, помидоры, лимонный сок, оливковое масл...",легко
